In [1]:
using MySQL
using DataFrames
using Dates
using TimeZones
using StatsBase, LinearAlgebra
using LaTeXStrings
using JSON
using CSV

import Base: @kwdef

In [4]:
timestamp(t; target_tz::VariableTimeZone=tz"America/Los_Angeles") = astimezone(
    ZonedDateTime(Dates.unix2datetime(t), tz"GMT"), target_tz
)

function get_emeter_data(conn::MySQL.Connection, name::String, idx::Int64)
    DBInterface.execute(
        DBInterface.prepare(conn,
            "SET @v1 := (SELECT emeter_start FROM $(name) WHERE idx = $(idx))"
        )
    )
    DBInterface.execute(
        DBInterface.prepare(conn,
            "SET @v2 := (SELECT emeter_end FROM $(name) WHERE idx = $(idx))"
        )
    )
    events = DBInterface.execute(
        DBInterface.prepare(conn,
            "SELECT * FROM $(name)_emeter WHERE idx BETWEEN @v1 and @v2"
        )
    ) |> DataFrame
    events = transform(
        events,
        :t => (x->timestamp.(x)) => :timestamp
    )
    return events
end

@kwdef mutable struct DBCredentials
    host::String
    name::String
    username::String
    password::String
end

DBCredentials

In [5]:
db_credenitals = DBCredentials(
    ;JSON.parsefile(
        joinpath("..", "..", "coffee-secrets", "db.json"),
        dicttype=Dict{Symbol,String}
    )...
)
db_credenitals.host = "127.0.0.1";

In [6]:
db_conn = DBInterface.connect(
    MySQL.Connection, db_credenitals.host,
    db_credenitals.username, db_credenitals.password,
    db=db_credenitals.name
)

MySQL.Connection(host="127.0.0.1", user="coffee", port="3306", db="coffee")

In [7]:
name = "Arstotzka_Ministry_of_Energy"
idx_max = 54
idx_blacklist = [
    1,2,3,4,5,6,7,8,9,  # Debugging
    51  #  Battery Charger
]

all_emeter_data = DataFrame[]

for idx=1:idx_max
    if idx in idx_blacklist
        continue
    end
    push!(all_emeter_data, get_emeter_data(db_conn, name, idx))
end
all_emeter_data = vcat(all_emeter_data...)

,idx,t,ma,mv,mw,wh,timestamp
,Int32,Float64?,Int32?,Int32?,Int32?,Int32?,ZonedDat…
1,12,1.64349e9,59,121266,7139,7512,2022-01-29T12:19:41.336-08:00
2,13,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:45.166-08:00
3,14,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:53.324-08:00
4,15,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:54.646-08:00
5,16,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:55.542-08:00
6,17,1.64349e9,59,121266,7139,7512,2022-01-29T12:20:56.556-08:00
7,18,1.64349e9,59,121266,7139,7512,2022-01-29T12:21:03.581-08:00
8,19,1.64349e9,59,121266,7139,7512,2022-01-29T12:21:16.887-08:00
9,20,1.64349e9,59,121266,7139,7512,2022-01-29T12:21:20.293-08:00


In [8]:
all_emeter_data[end,:]

,idx,t,ma,mv,mw,wh,timestamp
,Int32,Float64?,Int32?,Int32?,Int32?,Int32?,ZonedDat…
1551562,1559921,1.64548e9,1566,119911,167817,23342,2022-02-21T12:35:34.350-08:00


In [13]:
function coarsegrained(data, factor)
    df = DataFrame(
        t_start=Float64[], t_end=Float64[], 
        timestamp_start=ZonedDateTime[], timestamp_end=ZonedDateTime[],
        ma=Float64[], mv=Float64[], mw=Float64[],
        wh_start=Int64[], wh_end=Int64[]
    )

    for i=1:factor:nrow(data)-factor
        section  = @view data[i:i+factor, :]
        t_start  = section[1, [:t, :timestamp]]
        t_end    = section[end, [:t, :timestamp]]
        wh_start = section[1, :wh]
        wh_end   = section[end, :wh]
        means    = mean.(eachcol(section[!, [:ma, :mv, :mw]]))
        push!(df, (
                t_start[:t], t_end[:t], t_start[:timestamp], t_end[:timestamp],
                means[1], means[2], means[3],
                wh_start, wh_end
            )
        )
    end
    return df
end

coarsegrained (generic function with 1 method)

In [14]:
df_10seconds = coarsegrained(all_emeter_data, 10)
df_minuntes = coarsegrained(all_emeter_data, 60)
df_10minuntes = coarsegrained(all_emeter_data, 600)
df_hours = coarsegrained(all_emeter_data, 60*60)

,t_start,t_end,timestamp_start,timestamp_end
,Float64,Float64,ZonedDat…,ZonedDat…
1,1.64349e9,1.64352e9,2022-01-29T12:19:41.336-08:00,2022-01-29T20:14:11.100-08:00
2,1.64352e9,1.64352e9,2022-01-29T20:14:11.100-08:00,2022-01-29T21:57:39.900-08:00
3,1.64352e9,1.64353e9,2022-01-29T21:57:39.900-08:00,2022-01-29T23:28:01.599-08:00
4,1.64353e9,1.64353e9,2022-01-29T23:28:01.599-08:00,2022-01-30T00:30:42.107-08:00
5,1.64353e9,1.64354e9,2022-01-30T00:30:42.107-08:00,2022-01-30T01:32:23.883-08:00
6,1.64354e9,1.64354e9,2022-01-30T01:32:23.883-08:00,2022-01-30T02:35:43.941-08:00
7,1.64354e9,1.64354e9,2022-01-30T02:35:43.941-08:00,2022-01-30T03:38:55.029-08:00
8,1.64354e9,1.64355e9,2022-01-30T03:38:55.029-08:00,2022-01-30T04:41:03.245-08:00
9,1.64355e9,1.64355e9,2022-01-30T04:41:03.245-08:00,2022-01-30T05:43:46.964-08:00


In [15]:
CSV.write("fridge_10seconds.csv", df_10seconds)
CSV.write("fridge_minutes.csv", df_minuntes)
CSV.write("fridge_10minutes.csv", df_10minuntes)
CSV.write("fridge_hours.csv", df_hours)

"fridge_hours.csv"